In [1]:
from MCTD import MCTD
from checkers import Game, RandomPlayer
from cengines import Easy

In [ ]:
p1 = MCTD.from_file("Game100")
p2 = MCTD.from_file("Game300")
game = Game()
game.simulate(p1, p2)

In [2]:
players = {
    "bootstrapped" : MCTD.from_file("bootstrapped_100000_10000"),
    "endgame" : MCTD.from_file("Endgame10"),
    "100games" : MCTD.from_file("Game100"),
    "200games" : MCTD.from_file("Game200"),
    "300games" : MCTD.from_file("Game300"),
    "random" : RandomPlayer(),
    "easy" : Easy()
}

In [ ]:
results = [['x' for _ in range(7)] for _ in range(7)]
for i, p1 in enumerate(players.values()):
    for j, p2 in enumerate(players.values()):
        if i == j:
            continue
        else:
            game = Game()
            game.simulate(p1, p2, 0.5, rendering=True)
            results[i][j] = f"{(game.result + 1)/2}"

In [33]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Table(
  header = dict(
    values = ['<b>Results</b>'] + [k for k in players.keys()],
    line_color='darkslategray',
    fill_color='green',
    align=['left','center'],
    font=dict(color='white', size=12),
    height=40,
  ),
  cells=dict(
    values= [[k for k in players.keys()]] + [[results[i][j] for i in range(7)] for j in range(7)],
    line_color='darkslategray',
    fill=dict(color=['green', 'white']),
    align=['left', 'center'],
    font_size=12,
    height=30,)
    ),
])
fig.update_layout(width=1000, height=500)
fig.show()